# Building a basic LLM Agent using Langgraph

In [28]:
# From https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/

In [29]:
import openai
import re
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing import Literal
from langchain_core.tools import tool

_ = load_dotenv()

## 1. Setup the LLM

In [30]:
#
# Params
#
OPENAI_MODEL_NAME = "gpt-4o-mini"
OPENAI_MODEL_TEMPERATURE = 0

#
# Initialize OpenAI
#
llm = ChatOpenAI(model=OPENAI_MODEL_NAME, temperature=OPENAI_MODEL_TEMPERATURE)

## 2. Creating the Agent

### 2.1 Define the tools

In [31]:
#
# Define the tools that will be available to the agent. The description of each tool will be used by the agent to understand what the tool does.
#

@tool
def calculate(formula):
    """Use this to run math calculations. For example, calculate: 4 * 7 / 3"""
    return eval(formula)

tools = [calculate]

### 2.2 Create the graph

In [40]:
prompt = """
# Focus

# Steps


# Notes
- Answer using the same language as the customer.
- Avoid making assumptions, ask questions to clarify any doubts.
""".strip()

In [33]:
from langchain_core.messages.system import SystemMessage
from langgraph.checkpoint.memory import MemorySaver

#
# The built-in create_react_agent function embeds all the logic needed to create a basic ReAct agent
#
memory = MemorySaver()

graph = create_react_agent(model=llm, tools=tools, state_modifier=prompt, checkpointer=memory, debug=False)

In [34]:
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [35]:
#
# Define an utility function to print the stream of messages
#
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


def ask_assistant(question: str, config: dict = None):
    inputs = {"messages": [("user", question)]}
    print_stream(graph.stream(inputs, config=config, stream_mode="values"))

In [36]:
config = {"configurable": {"thread_id": "1"}}
question = "Desidero spedire una grande scultura delicata che misura circa 244 cm di altezza"
inputs = {"messages": [("user", question)]}
graph.invoke(inputs, config=config, stream_mode="values")

{'messages': [HumanMessage(content='Desidero spedire una grande scultura delicata che misura circa 244 cm di altezza', additional_kwargs={}, response_metadata={}, id='0fe340a5-ced1-47c1-bc66-c15fd8bab82e'),
  AIMessage(content='Thought: La scultura è alta 244 cm, quindi dovrò considerare container che possano ospitare oggetti alti. Dovrei chiedere ulteriori dettagli sulla larghezza, profondità e peso della scultura, oltre a sapere se ci sono requisiti specifici per la protezione o il trasporto. \n\nChiederò quindi informazioni aggiuntive sulla larghezza, profondità e peso della scultura.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 1991, 'total_tokens': 2081, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerp

## 3. Execute the Agent

In [37]:
# Set conversation thread
config = {"configurable": {"thread_id": "1"}}

In [ ]:
question = """

""".strip()

In [38]:
ask_assistant(question, config=config)

================================ Human Message =================================

Desidero spedire una grande scultura delicata che misura circa 244 cm di altezza e necessita di spazio extra oltre a una manipolazione sicura, mantenendo una temperatura controllata; quale container sarebbe il più adatto per questa spedizione?
================================== Ai Message ==================================

Thought: La scultura è alta 244 cm e richiede spazio extra e una manipolazione sicura, oltre a un controllo della temperatura. Dato che la scultura è delicata, un container refrigerato potrebbe essere la scelta migliore. Tuttavia, devo anche considerare le dimensioni e il peso della scultura per assicurarmi che possa adattarsi al container. 

Chiederò quindi informazioni sul peso della scultura e sulle sue dimensioni in larghezza e profondità per determinare il container più adatto.


In [12]:
# cli for chat using a while loop
while True:
    question = input("You: ")
    if question.startswith("/bye") or not question.strip():
        break
    ask_assistant(question, config=config)
    print("\n")